In [1]:
import pandas as pd
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestClassifier
import catboost as cat
from catboost import CatBoostClassifier
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import log_loss, confusion_matrix, accuracy_score
import xgboost as xgb
import lightgbm as lgb
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [2]:
#importing files
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample = pd.read_csv('sample_submission.csv')

In [3]:
def fill_missing_values(data):
    
    '''
    Function to input missing values based on the column object type
    '''
    
    cols = list(data.columns)
    for col in cols:
        if data[col].dtype == 'int64' or data[col].dtype == 'float64':
        
            data[col] = data[col].fillna(data[col].mean())
        
        #elif data[col].dtype == 'O' or data[col].dtype == 'object':
        #    data[col] = data[col].fillna(data[col].mode()[0])
            
        else:
            data[col] = data[col].fillna(data[col].mode()[0])
            
    return data

def freq_encoding(traindata, testdata, *args):
    
    for _ in args:
        train_enc = (traindata.groupby(_).size()) / len(traindata)
        test_enc = (testdata.groupby(_).size()) / len(testdata)
        
        traindata[_] = traindata[_].apply(lambda x: train_enc[x])
        testdata[_] = testdata[_].apply(lambda x: test_enc[x])
    
    return traindata, testdata

def one_hot_encoding(traindata, testdata, *args):
    
    for ii in args:
        traindata = pd.get_dummies(traindata, prefix=[ii], columns=[ii])
        testdata = pd.get_dummies(testdata, prefix=[ii], columns=[ii])
        
    return traindata, testdata

def drop_columns(traindata, testdata, target, *args):
    
    labels = np.array(traindata[target])
    
    columns = []
    for _ in args:
        columns.append(_)
        
    traindata = traindata.drop(columns, axis=1)
    traindata = traindata.drop(target, axis=1)
    testdata = testdata.drop(columns, axis=1)
        
    return labels, traindata, testdata

In [4]:
cols = ['ZONE1', 'ZONE2', 'TIGO', 'user_id', 'MRG']

labels, train1, test1 = drop_columns(train, test, 'CHURN', *cols)

one_hot_cols = ['REGION', 'TENURE']

train1, test1 = one_hot_encoding(train1, test1, *one_hot_cols)

In [5]:
train1 = train1.fillna(-999)
test1 = test1.fillna(-999)

In [6]:
freq_col = ['TOP_PACK']

train1, test1 = freq_encoding(train1, test1, *freq_col)

In [7]:
features = np.array(train1)
test_enc = np.array(test1)

#scaler = preprocessing.StandardScaler().fit(features)
#features = scaler.transform(features)
#test_enc = scaler.transform(test_enc)

In [8]:
#using StandardScaler function to scale the numeric features 

scaler = preprocessing.StandardScaler().fit(features[:, :])
features[:, :] = scaler.transform(features[:, :])
features[:, :]

array([[ 2.11347837,  0.77723114,  2.22374343, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293],
       [ 0.16341176,  0.77101117,  0.16829127, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293],
       [-0.26652419,  0.7171048 , -0.27496434, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293],
       ...,
       [-0.38936303,  0.71503147, -0.39626525, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293],
       [-0.65024202, -1.36036399, -0.50075666, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293],
       [-0.65024202, -1.36036399, -0.65340499, ..., -0.1456742 ,
        -0.0764623 ,  0.23278293]])

In [9]:
#using StandardScaler function to scale the numeric features 

scaler1 = preprocessing.StandardScaler().fit(test_enc[:, :])
test_enc[:, :] = scaler1.transform(test_enc[:, :])
test_enc[:, :]

array([[-0.65111099, -1.36390268, -0.65551124, ..., -0.14692214,
        -0.07618062,  0.23361125],
       [-0.65111099, -1.36390268, -0.50292553, ..., -0.14692214,
        -0.07618062,  0.23361125],
       [-0.20772843,  0.73999703, -0.15677221, ..., -0.14692214,
        -0.07618062,  0.23361125],
       ...,
       [-0.22302276,  0.72339822, -0.23238455, ...,  6.80632622,
        -0.07618062, -4.28061578],
       [-0.11596247,  0.71717366, -0.12410768, ..., -0.14692214,
        -0.07618062,  0.23361125],
       [-0.65111099, -1.36390268, -0.65551124, ..., -0.14692214,
        -0.07618062,  0.23361125]])

In [10]:
## Randomly splitting into train and test data
nr.seed(9988)
indx = range(features.shape[0])
indx = ms.train_test_split(indx, test_size = 0.3)
x_train = features[indx[0],:]
y_train = np.ravel(labels[indx[0]])
x_test = features[indx[1],:]
y_test = np.ravel(labels[indx[1]])

In [11]:
'''
    Randomly splitting the test data into two parts; one for validation during training
    and the other for testing the model
'''

np.random.seed(9988)
indx1 = range(x_test.shape[0])
indx1 = ms.train_test_split(indx1, test_size = 0.5)
x_test1 = x_test[indx1[0],:]
y_test1 = np.ravel(y_test[indx1[0]])
x_test2 = x_test[indx1[1],:]
y_test2 = np.ravel(y_test[indx1[1]])

In [19]:
xg = xgb.XGBClassifier(objective='reg:logistic',
                       tree_method='gpu_hist')
xg

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='reg:logistic', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)

In [21]:
params = {'n_estimators': [3000],
          'max_depth': [10, 12, 14, 16, 18, 20]}

eval_set = [(x_train, y_train), (x_test1, y_test1)]
clf = GridSearchCV(xg, params)
clf.fit(x_train, y_train, eval_metric="logloss", eval_set=eval_set, verbose=2, early_stopping_rounds = 200)

[0]	validation_0-logloss:0.52543	validation_1-logloss:0.52511
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 200 rounds.
[2]	validation_0-logloss:0.37683	validation_1-logloss:0.37619
[4]	validation_0-logloss:0.31415	validation_1-logloss:0.31405
[6]	validation_0-logloss:0.28407	validation_1-logloss:0.28488


KeyboardInterrupt: 

In [12]:
xg = xgb.XGBRegressor(objective='reg:logistic',
                       boosting_type='gbtree', 
                       learning_rate=0.01, 
                       n_estimators=5000, 
                       max_depth=12, 
                       base_score=0.7,
                       max_features='auto'
                     )

eval_set = [(x_train, y_train), (x_test1, y_test1)]
xg.fit(x_train, y_train, eval_metric="logloss", eval_set=eval_set, verbose=2, early_stopping_rounds = 200)

[00:25:41] WARNING: /workspace/src/learner.cc:480: 
Parameters: { boosting_type, max_features, prediction } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-logloss:1.02950	validation_1-logloss:1.03031
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 200 rounds.
[2]	validation_0-logloss:0.99987	validation_1-logloss:1.00048


KeyboardInterrupt: 